In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils

from keras.models import model_from_yaml

import matplotlib.pyplot as plt

nclasses = 10

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nclasses)
Y_test = np_utils.to_categorical(y_test, nclasses)

model = Sequential()
model.add(Dense(100, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

batch_size = 128
nb_epoch = 10

model.fit(X_train, Y_train,
          batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

loaded_model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

score = loaded_model.evaluate(X_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])


first_sample = X_test[0].reshape((1,784))
pred = loaded_model.predict(first_sample)
print ( pred )

plt.imshow(first_sample.reshape(28,28), cmap='gray')
plt.show()

60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
469/469 [==============================] - 2s 4ms/step - loss: 0.3790 - accuracy: 0.8957 - val_loss: 0.2082 - val_accuracy: 0.9401
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 0.1806 - accuracy: 0.9489 - val_loss: 0.1496 - val_accuracy: 0.9570
Epoch 3/10
469/469 [==============================] - 3s 6ms/step - loss: 0.1313 - accuracy: 0.9625 - val_loss: 0.1203 - val_accuracy: 0.9637
Epoch 4/10
469/469 [=============================

/home/joaomcouto/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/saving/model_config.py:88: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object/apply:tensorflow.python.framework.tensor_shape.TensorShape'
  in "<unicode string>", line 4, column 22:
      build_input_shape: !!python/object/apply:tensorflow ... 
                         ^